In [1]:
import os
import math
import random
import numpy as np
import re
import dspy
from dspy.teleprompt import *
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

os.environ['TOGETHER_API_KEY'] = '35ba5bebf6288e43fdc8989965161592e3335d7067c772c0c6995cdc0e60cd88'
os.environ['TOGETHER_API_BASE'] = 'https://api.together.xyz/v1'

In [25]:
# constants
NUM_PAIRS = 1
NUM_CITIES = 10
TRAIN_INSTANCES = 100
TEST_INSTANCES = 100
# CITIES = " ".join(map(str, list(np.arange(NUM_CITIES))))
CITIES = ", ".join(map(str, list(np.arange(NUM_CITIES))))
NUM_THREADS = 5
K = 6

In [26]:
CITIES

'0, 1, 2, 3, 4, 5, 6, 7, 8, 9'

In [3]:
# OR Tools
def create_data_model(distance_matrix, constraints):
    """Stores the data for the problem."""
    data = {
        "distance_matrix": distance_matrix,
        "pickups_deliveries": constraints,
        "num_vehicles": 1,
        "depot": 0
    }
    return data

def print_solution(data, manager, routing, solution):
    """Prints solution on console and returns the route and distance."""
    total_distance = 0
    optimal_route = []
    
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        route = []
        route_distance = 0
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route.append(node_index)
            plan_output += f" {node_index} -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        node_index = manager.IndexToNode(index)
        route.append(node_index)
        plan_output += f"{node_index}\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        # print(plan_output)
        optimal_route.append(route)
        total_distance += route_distance
    optimal_route = optimal_route[0][:-1]
    # print(f"Total Distance of all routes: {total_distance}m")
    return optimal_route, total_distance


def solve_pdp_with_constraints(locations, constraints, distance_matrix, num_vehicles=1, depot=0):
    """Solve the PDP using OR-Tools."""
    locations = [list(loc) for loc in locations]
    distance_matrix = distance_matrix.astype(int).tolist()

    data = create_data_model(distance_matrix, constraints)

    manager = pywrapcp.RoutingIndexManager(len(data["distance_matrix"]), num_vehicles, depot)

    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    dimension_name = "Distance"
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        3000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name,
    )
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    for request in data["pickups_deliveries"]:
        pickup_index = manager.NodeToIndex(request[0])
        delivery_index = manager.NodeToIndex(request[1])
        routing.AddPickupAndDelivery(pickup_index, delivery_index)
        routing.solver().Add(routing.VehicleVar(pickup_index) == routing.VehicleVar(delivery_index))
        routing.solver().Add(distance_dimension.CumulVar(pickup_index) <= distance_dimension.CumulVar(delivery_index))

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION

    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        optimal_route, total_distance = print_solution(data, manager, routing, solution)
        return optimal_route, total_distance
    else:
        print("No solution found!")
        return None

In [43]:
def euclidean_distance(point1, point2):
    return round(math.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2), 1)

def calc_path_distance(path, distances):
    total_distance = 0
    if len(path) < 2:
        return 0
    for i in range(len(path) - 1):
        total_distance += distances[path[i]][path[i + 1]]
    total_distance += distances[path[-1]][path[0]]
    return total_distance

def make_graphs(num_instances, num_cities):
    x_range = (-20, 20)
    y_range = (-20, 20)

    distanceList = []
    precedence_constraints = []
    for _ in range(num_instances):
        coordinates = [(random.randint(*x_range), random.randint(*y_range)) for _ in range(num_cities)]
        distance_matrix = [[euclidean_distance(coordinates[i], coordinates[j]) for j in range(num_cities)] for i in range(num_cities)]
        
        # Generate NUM_PAIRS random precedence pairs
        # ensures there is no constraint where 0 is after sumn
        pairs = []
        for _ in range(NUM_PAIRS):
            a, b = random.sample(range(1, num_cities), 2)
            pairs.append((a, b))
        
        distanceList.append(np.array(distance_matrix))
        precedence_constraints.append(pairs)
    return coordinates, distanceList, precedence_constraints

def make_dataset(coordinates, distanceList, precedence_constraints):
    dataset = []
    for i in range(len(distanceList)):
        matrix = distanceList[i]
        constraints = precedence_constraints[i]
        # Replace with your own PDP solver or use OR-tools for a quick setup
        optimal_route, total_distance = solve_pdp_with_constraints(locations=coordinates, constraints=constraints, distance_matrix=matrix)
        data_point = {
            "distances": matrix.tolist(),
            "route": optimal_route,
            "optimal_distance": total_distance,
            "constraints": constraints
        }
        dataset.append(data_point)
    return dataset

def adjacency_matrix_to_distances(matrix):
    distances_str = ""
    num_cities = len(matrix)
    for i in range(num_cities):
        for j in range(i + 1, num_cities):
            distances_str += f"  - {i} to {j}: {matrix[i][j]}\n"
    return distances_str

def makeDSPYExamples(dataset):
    exampleList = []
    for example in dataset:
        str_distances = adjacency_matrix_to_distances(example["distances"])
        route = " -> ".join(map(str, example["route"]))
        constraints = ", ".join([f"({p[0]}, {p[1]})" for p in example["constraints"]])
        opt_dis = str(example["optimal_distance"])
        exampleObj = dspy.Example(Cities=CITIES, Distances=str_distances, Constraints=constraints, Route=route, Optimal_Distance=opt_dis).with_inputs("Cities", "Distances", "Constraints")
        exampleList.append(exampleObj)
    return exampleList

def random_baseline(distances, constraints):
    numbers = list(range(1, NUM_CITIES))
    random.shuffle(numbers)
    numbers.insert(0, 0)
    for constraint in constraints:
        a, b = constraint
        if numbers.index(a) > numbers.index(b):
            numbers.remove(a)
            numbers.insert(numbers.index(b), a)
    
    path_length = calc_path_distance(path=numbers, distances=distances)
    return numbers, path_length

In [44]:
# Train set:
train_coordinates, train_dl, train_p = make_graphs(TRAIN_INSTANCES, NUM_CITIES)
train_ds = make_dataset(train_coordinates, train_dl, train_p)
pdp_trainset = makeDSPYExamples(train_ds)

In [45]:
pdp_trainset[0]

Example({'Cities': '0, 1, 2, 3, 4, 5, 6, 7, 8, 9', 'Distances': '  - 0 to 1: 27.0\n  - 0 to 2: 25.5\n  - 0 to 3: 5.7\n  - 0 to 4: 14.8\n  - 0 to 5: 24.2\n  - 0 to 6: 21.6\n  - 0 to 7: 21.2\n  - 0 to 8: 12.0\n  - 0 to 9: 28.7\n  - 1 to 2: 11.7\n  - 1 to 3: 23.3\n  - 1 to 4: 36.4\n  - 1 to 5: 13.4\n  - 1 to 6: 38.3\n  - 1 to 7: 31.9\n  - 1 to 8: 28.6\n  - 1 to 9: 28.2\n  - 2 to 3: 24.2\n  - 2 to 4: 38.4\n  - 2 to 5: 2.2\n  - 2 to 6: 29.7\n  - 2 to 7: 22.4\n  - 2 to 8: 22.0\n  - 2 to 9: 16.6\n  - 3 to 4: 14.2\n  - 3 to 5: 23.3\n  - 3 to 6: 26.6\n  - 3 to 7: 25.0\n  - 3 to 8: 16.3\n  - 3 to 9: 30.7\n  - 4 to 5: 37.5\n  - 4 to 6: 34.1\n  - 4 to 7: 35.4\n  - 4 to 8: 26.2\n  - 4 to 9: 43.4\n  - 5 to 6: 27.5\n  - 5 to 7: 20.1\n  - 5 to 8: 20.0\n  - 5 to 9: 14.8\n  - 6 to 7: 8.0\n  - 6 to 8: 10.8\n  - 6 to 9: 19.4\n  - 7 to 8: 9.2\n  - 7 to 9: 11.7\n  - 8 to 9: 18.4\n', 'Constraints': '(4, 2)', 'Route': '0 -> 4 -> 3 -> 1 -> 2 -> 5 -> 9 -> 7 -> 6 -> 8', 'Optimal_Distance': '119'}) (input_keys={'

In [46]:
# Test set:
test_coordinates, test_dl, test_p = make_graphs(TEST_INSTANCES, NUM_CITIES)
test_ds = make_dataset(test_coordinates, test_dl, test_p)
pdp_testset = makeDSPYExamples(test_ds)

In [47]:
llama = dspy.Together(model="meta-llama/Meta-Llama-3-70B", max_tokens=50)
dspy.configure(lm=llama)

In [48]:
pdp_trainset[0]

Example({'Cities': '0, 1, 2, 3, 4, 5, 6, 7, 8, 9', 'Distances': '  - 0 to 1: 27.0\n  - 0 to 2: 25.5\n  - 0 to 3: 5.7\n  - 0 to 4: 14.8\n  - 0 to 5: 24.2\n  - 0 to 6: 21.6\n  - 0 to 7: 21.2\n  - 0 to 8: 12.0\n  - 0 to 9: 28.7\n  - 1 to 2: 11.7\n  - 1 to 3: 23.3\n  - 1 to 4: 36.4\n  - 1 to 5: 13.4\n  - 1 to 6: 38.3\n  - 1 to 7: 31.9\n  - 1 to 8: 28.6\n  - 1 to 9: 28.2\n  - 2 to 3: 24.2\n  - 2 to 4: 38.4\n  - 2 to 5: 2.2\n  - 2 to 6: 29.7\n  - 2 to 7: 22.4\n  - 2 to 8: 22.0\n  - 2 to 9: 16.6\n  - 3 to 4: 14.2\n  - 3 to 5: 23.3\n  - 3 to 6: 26.6\n  - 3 to 7: 25.0\n  - 3 to 8: 16.3\n  - 3 to 9: 30.7\n  - 4 to 5: 37.5\n  - 4 to 6: 34.1\n  - 4 to 7: 35.4\n  - 4 to 8: 26.2\n  - 4 to 9: 43.4\n  - 5 to 6: 27.5\n  - 5 to 7: 20.1\n  - 5 to 8: 20.0\n  - 5 to 9: 14.8\n  - 6 to 7: 8.0\n  - 6 to 8: 10.8\n  - 6 to 9: 19.4\n  - 7 to 8: 9.2\n  - 7 to 9: 11.7\n  - 8 to 9: 18.4\n', 'Constraints': '(4, 2)', 'Route': '0 -> 4 -> 3 -> 1 -> 2 -> 5 -> 9 -> 7 -> 6 -> 8', 'Optimal_Distance': '119'}) (input_keys={'

In [88]:
def check_order(route, pickup, delivery):
    return route.index(pickup) < route.index(delivery)

class PDP(dspy.Module):
    def __init__(self):
        super().__init__()
        self.make_route = dspy.Predict(PDPSignature)
        
    def forward(self, Cities, Distances, Constraints):
        pred_route = self.make_route(Cities=Cities, Distances=Distances, Constraints=Constraints)
        numbers, pickup, delivery = extract_route(pred_route.Route, Constraints)
        dspy.Suggest(
                check_order(numbers, pickup, delivery),
                f"{pickup} must be visited before {delivery}"
            )
        return pred_route
    
class PDPSignature(dspy.Signature):
    """Generate a TSP route starting at city 0 that visits the specified pickup before the delivery node, minimizing distance traveled."""
    Cities = dspy.InputField(desc="List of city indices to visit")
    Distances = dspy.InputField(desc="Distances between two cities")
    Route = dspy.OutputField(desc="Optimized route visiting all cities with pickup and delivery")
    Constraints = dspy.InputField(desc="Tuple containing (pickup, delivery) nodes")


def extract_route(route, constraints, N=NUM_CITIES):
    # Extract the first N numbers from the route string
    numbers = re.findall(r'\d+', route)[:N]
    
    # Convert the numbers to integers
    numbers = list(map(int, numbers))

    pattern = r'\((\d+),\s*(\d+)\)'

    # Use re.search to find the numbers
    match = re.search(pattern, constraints)
    
    if match:
        pickup = int(match.group(1))
        delivery = int(match.group(2))
        # print("First number:", pickup)
        # print("Second number:", delivery)
    else:
        print("No match found")
    return numbers, pickup, delivery

def eval_tour(cities, route, distances, constraints):
    try:
        route, pickup, delivery = extract_route(route, constraints)  # make it a list of ints
    except ValueError:
        raise ValueError(f"Invalid route: {route}")

    # Check precedence constraints
    if route.index(pickup) > route.index(delivery):
        raise ValueError(f"Precedence constraint violated: {pickup} must be visited before {delivery}")

    distances_dict = {}
    pattern = re.compile(r"\s*-\s*(\d+)\s*to\s*(\d+):\s*([\d.]+)")
    
    for line in distances.splitlines():
        match = pattern.match(line)
        if match:
            city1, city2, distance = match.groups()
            city1, city2, distance = int(city1), int(city2), float(distance)
            distances_dict[(city1, city2)] = distance
            distances_dict[(city2, city1)] = distance  # since the distances are symmetrical
    
    # Step 2: Calculate the total distance of the route
    total_distance = 0.0
    for i in range(len(route) - 1):
        city1 = route[i]
        city2 = route[i + 1]
        total_distance += distances_dict[(city1, city2)]
    
    # To return to the starting point
    total_distance += distances_dict[(route[-1], route[0])]
    return total_distance

# Validation function for the Precedence-Constrained TSP
def metric(example, pred, trace=None):
    try:
        distance = eval_tour(example.Cities, pred.Route, example.Distances, example.Constraints)
        return -distance  # Return negative distance to maximize the metric
    except ValueError as e:
        dspy.logger.error(e)
        return -200

In [89]:
from dspy.primitives.assertions import backtrack_handler

# Transform the module to include the backtracking mechanism
baleen_with_suggestions = PDP().activate_assertions(backtrack_handler)

teleprompter = LabeledFewShot(k=K)
compiled_pdp = teleprompter.compile(baleen_with_suggestions, trainset=pdp_trainset)

evaluater = Evaluate(devset=pdp_testset, metric=metric, num_threads=NUM_THREADS, display_progress=True, display_table=0)
evaluater(compiled_pdp)

Average Metric: -4212.8 / 26  (-16203.1):  26%|▎| 26/100 [01:23<02:57,  2.39s/it2024-08-05T18:18:11.357826Z [error    ] Precedence constraint violated: 7 must be visited before 2 [__main__] filename=1988560602.py lineno=85
Average Metric: -6105.9000000000015 / 37  (-16502.4):  37%|▎| 37/100 [01:54<03:12024-08-05T18:18:42.345965Z [error    ] Precedence constraint violated: 6 must be visited before 7 [__main__] filename=1988560602.py lineno=85
Average Metric: -6602.500000000002 / 40  (-16506.3):  40%|▍| 40/100 [02:04<02:512024-08-05T18:18:51.844585Z [error    ] Precedence constraint violated: 6 must be visited before 9 [__main__] filename=1988560602.py lineno=85
Average Metric: -9049.200000000003 / 55  (-16453.1):  55%|▌| 55/100 [02:47<01:512024-08-05T18:19:35.529138Z [error    ] Precedence constraint violated: 6 must be visited before 1 [__main__] filename=1988560602.py lineno=85
Average Metric: -12134.5 / 73  (-16622.6):  73%|▋| 73/100 [03:42<01:35,  3.55s/i2024-08-05T18:20:30.213213Z 

KeyboardInterrupt: 

2024-08-05T18:20:44.328134Z [error    ] Precedence constraint violated: 3 must be visited before 5 [__main__] filename=1988560602.py lineno=85


In [90]:
llama.inspect_history(n=1)




Generate a TSP route starting at city 0 that visits the specified pickup before the delivery node, minimizing distance traveled.

---

Cities: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9
Distances: - 0 to 1: 13.6 - 0 to 2: 24.6 - 0 to 3: 10.6 - 0 to 4: 10.0 - 0 to 5: 22.8 - 0 to 6: 23.1 - 0 to 7: 24.2 - 0 to 8: 41.0 - 0 to 9: 11.7 - 1 to 2: 22.0 - 1 to 3: 3.2 - 1 to 4: 21.1 - 1 to 5: 11.2 - 1 to 6: 13.4 - 1 to 7: 11.2 - 1 to 8: 29.0 - 1 to 9: 7.6 - 2 to 3: 22.8 - 2 to 4: 22.0 - 2 to 5: 17.0 - 2 to 6: 13.0 - 2 to 7: 21.0 - 2 to 8: 26.9 - 2 to 9: 28.4 - 3 to 4: 18.8 - 3 to 5: 14.0 - 3 to 6: 15.8 - 3 to 7: 14.3 - 3 to 8: 32.1 - 3 to 9: 5.7 - 4 to 5: 27.2 - 4 to 6: 25.9 - 4 to 7: 29.7 - 4 to 8: 44.3 - 4 to 9: 21.4 - 5 to 6: 4.1 - 5 to 7: 4.0 - 5 to 8: 18.2 - 5 to 9: 18.7 - 6 to 7: 8.1 - 6 to 8: 18.4 - 6 to 9: 21.0 - 7 to 8: 18.0 - 7 to 9: 18.0 - 8 to 9: 36.1
Constraints: (9, 5)
Route: 0 -> 9 -> 3 -> 1 -> 7 -> 5 -> 6 -> 8 -> 2 -> 4

Cities: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9
Distances: - 0 to 1: 28.0 - 0 to 

'\n\n\nGenerate a TSP route starting at city 0 that visits the specified pickup before the delivery node, minimizing distance traveled.\n\n---\n\nCities: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9\nDistances: - 0 to 1: 13.6 - 0 to 2: 24.6 - 0 to 3: 10.6 - 0 to 4: 10.0 - 0 to 5: 22.8 - 0 to 6: 23.1 - 0 to 7: 24.2 - 0 to 8: 41.0 - 0 to 9: 11.7 - 1 to 2: 22.0 - 1 to 3: 3.2 - 1 to 4: 21.1 - 1 to 5: 11.2 - 1 to 6: 13.4 - 1 to 7: 11.2 - 1 to 8: 29.0 - 1 to 9: 7.6 - 2 to 3: 22.8 - 2 to 4: 22.0 - 2 to 5: 17.0 - 2 to 6: 13.0 - 2 to 7: 21.0 - 2 to 8: 26.9 - 2 to 9: 28.4 - 3 to 4: 18.8 - 3 to 5: 14.0 - 3 to 6: 15.8 - 3 to 7: 14.3 - 3 to 8: 32.1 - 3 to 9: 5.7 - 4 to 5: 27.2 - 4 to 6: 25.9 - 4 to 7: 29.7 - 4 to 8: 44.3 - 4 to 9: 21.4 - 5 to 6: 4.1 - 5 to 7: 4.0 - 5 to 8: 18.2 - 5 to 9: 18.7 - 6 to 7: 8.1 - 6 to 8: 18.4 - 6 to 9: 21.0 - 7 to 8: 18.0 - 7 to 9: 18.0 - 8 to 9: 36.1\nConstraints: (9, 5)\nRoute: 0 -> 9 -> 3 -> 1 -> 7 -> 5 -> 6 -> 8 -> 2 -> 4\n\nCities: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9\nDistances: - 0 to 1

In [ ]:
test_example = pdp_testset[0]
numerical_test_example = test_ds[4]
print(numerical_test_example)

In [ ]:
predicted_result = compiled_pdp(cities=test_example.cities, distances=test_example.distances, constraints=test_example.constraints)

predicted_route = predicted_result.route

predicted_distance = eval_tour(test_example.cities, predicted_route, test_example.distances, test_example.constraints)

print(f"Predicted route: {predicted_route}")
print(f"Total distance of the predicted route: {predicted_distance}")

optimal_route = test_example.route
optimal_distance = eval_tour(test_example.cities, optimal_route, test_example.distances, test_example.constraints)
print(f"Optimal route: {optimal_route}")
print(f"Total distance of the optimal route: {optimal_distance}")

In [ ]:
print(numerical_test_example)
path, distance = random_baseline(numerical_test_example["distance_matrix"], numerical_test_example["constraints"])
print(f"path is {path}")
print(f"distance is {distance}")

Random baseline eval:

In [ ]:
total_dis = 0
for i in range(TEST_INSTANCES):
    curr_example = test_ds[i]
    _, distance = random_baseline(curr_example["distance_matrix"], curr_example["constraints"])
    total_dis += distance
print(f"(RANDOM) total distance is {total_dis}")
print(f"(RANDOM) average distance is {total_dis/TEST_INSTANCES}")

In [ ]:
zs_TSP = evaluater(PDP())
print(f"(Zero Shot) average distance is {zs_PDP / len(pdp_testset)}")

Model eval:

In [ ]:
print("(MODEL) average distance is 159.6")

Optimal route eval:

In [ ]:
total_dis = 0
for i in range(TEST_INSTANCES):
    curr_example = test_ds[i]
    total_dis += curr_example["optimal_distance"]
print(f"(OPTIMAL) total distance is {total_dis}")
print(f"(OPTIMAL) average distance is {total_dis/TEST_INSTANCES}")